# lmで次の単語を予測して、性質を理解する

In [1]:
from torchtext import vocab, data
import fastai
import fire
import numpy as np


from fastai import *
from fastai.text import *
import torch
from fastai_contrib.utils import read_file, read_whitespace_file,\
    DataStump, validate, PAD, UNK, get_sentencepiece, PAD_TOKEN_ID

import pickle

from pathlib import Path

from collections import Counter

In [2]:
dir_path = 'data/wiki/ja-2/'
spm_dir = 'data/wiki/ja/'
lang='ja'
cuda_id=0
qrnn=True
subword=True
max_vocab=16000
bs=70
bptt=70
name='wt-2'
num_epochs=1
ds_pct=1.0

In [3]:
results = {}
model_dir = 'models' # removed from params, as it is absolute models location in train_clas and here it is relative
if not torch.cuda.is_available():
    print('CUDA not available. Setting device=-1.')
    cuda_id = -1
torch.cuda.set_device(cuda_id)

dir_path = Path(dir_path)
assert dir_path.exists()
model_dir = Path(model_dir)
model_dir.mkdir(exist_ok=True)

if spm_dir:
    spm_dir=Path(spm_dir)
else:
    spm_dir = dir_path
assert spm_dir.exists()

print('Batch size:', bs)
print('Max vocab:', max_vocab)
model_name = 'qrnn' if qrnn else 'lstm'
if qrnn:
    print('Using QRNNs...')

trn_path = dir_path / f'{lang}.wiki.train.tokens'
val_path = dir_path / f'{lang}.wiki.valid.tokens'
tst_path = dir_path / f'{lang}.wiki.test.tokens'
for path_ in [trn_path, val_path, tst_path]:
    assert path_.exists(), f'Error: {path_} does not exist.'

if subword:
    # apply sentencepiece tokenization
    trn_path = dir_path / f'{lang}.wiki.train.tokens'
    val_path = dir_path / f'{lang}.wiki.valid.tokens'

    read_file(trn_path, 'train')
    read_file(val_path, 'valid')
    
    # assume sentencepiece training is done after merge of wiki
    # here we're just loading the trained spm model
    sp = get_sentencepiece(spm_dir, None, 'wt-all', vocab_size=max_vocab)
    
    data_lm = TextLMDataBunch.from_csv(dir_path, 'train.csv',
                                        **sp)

In [4]:
stoi = sp['vocab'].stoi

In [5]:
itos = sp['vocab'].itos

In [6]:
emb_sz, nh, nl = 400, 1550, 3

In [7]:
import torch
from torch.autograd import Variable

In [8]:
torch.cuda.set_device(0)

In [9]:
learn = language_model_learner(
        data_lm, bptt=bptt, emb_sz=emb_sz, nh=nh, nl=nl, qrnn=qrnn,
        pad_token=PAD_TOKEN_ID,
        path = 'data/wiki/ja-100/',    
        model_dir= 'models',
        pretrained_fnames=['qrnn_wt-100', 'itos_wt-100'])

In [10]:
learn

LanguageLearner(data=TextLMDataBunch;
Train: LabelList
y: LMLabel (69293 items)
[Category 0, Category 0, Category 0, Category 0, Category 0]...
Path: data/wiki/ja-2
x: LMTextList (69293 items)
[Text ▁ x x bo s ▁ x x f l d ▁ 1, Text ▁ x x bo s ▁ x x f l d ▁ 1 ▁15 48 年 10 月 、 ジャン ヌ ・ ダル ブレ と 第 2 代 ヴァン ドーム 公 アント ワー ヌ が 結婚 した 。 カトリック の 優勢 な ヴァン ドーム に 、 短期間 ユ グ ノー が 滞在 した 。 15 62 年 、 ユ グ ノー が サン = ジョ ル ジュ 教会 を 汚 し 略奪 した 。 17 93 年には 、 城 の 心臓 部 にある ブル ボン = ヴァン ドーム 家の 真 の ネ クロ ポリス が 略奪 に遭い 、 現在は 廃 <unk> となっている 。 アンリ 4 世は 城 の 包囲 へ 向かい 、 158 9 年に ヴァン ドーム は カトリック 同盟 軍に 降伏 した 。, Text ▁ x x bo s ▁ x x f l d ▁ 1, Text ▁ x x bo s ▁ x x f l d ▁ 1 ▁16 23 年 、 ヴァン ドーム 公 セ ザール ・ ド ・ ブル ボン は オラ トリオ 会 の神 学校 を つく った 。 これが 現在の リ セー ・ ロン サール である 。, Text ▁ x x bo s ▁ x x f l d ▁ 1]...
Path: data/wiki/ja-2;
Valid: LabelList
y: LMLabel (17324 items)
[Category 0, Category 0, Category 0, Category 0, Category 0]...
Path: data/wiki/ja-2
x: LMTextList (17324 items)
[Text ▁ x x bo s ▁ x x f l d ▁ 1, Text ▁ x x bo s ▁ x

In [11]:
m = learn.model

In [12]:
s = '今日は'
s_toks = sp['tokenizer'].process_all([s])
s_toks = s_toks[0]
print(s_toks)
s_nums = [stoi.get(i, stoi[UNK]) for i in s_toks]
s_var = Variable(torch.from_numpy(np.array(s_nums)))[None]

['▁', '今日', 'は']


In [13]:
s_var[0]

tensor([   5, 2857,    6])

In [14]:
# Define the default tensor type at the top
torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available() 
                                                     else torch.FloatTensor)

In [15]:
def sample_model(m, s, l=50):
    s_toks = sp['tokenizer'].process_all([s])
    s_toks = s_toks[0]
    print(s_toks)
    s_nums = [stoi.get(i, stoi[UNK]) for i in s_toks]
#     s_var = V(np.array(s_nums))[None]
    s_var = Variable(torch.cuda.LongTensor(np.array(s_nums)))[None]
    
    m[0].bs=1
    m.eval()
    m.reset()

    res, *_ = m(s_var)
    print('...', end='')

    for i in range(l):
        r = torch.multinomial(res[-1].exp(), 2)
        #r = torch.topk(res[-1].exp(), 2)[1]
#         print(r)
        if r.data[0] == 0:
            r = r[1]
        else:
            r = r[0]
#         print(to_np(r))
        word = itos[to_np(r)]
#         print(r.unsqueeze(0))
        res, *_ = m(r.unsqueeze(0).unsqueeze(0))
        print(word, end=' ')
    m[0].bs=bs

In [16]:
device = torch.device('cuda:0')  # or 'cpu'
m = m.to(device)

In [17]:
sample_model(m,'料金が高い',l=200)

['▁', '料金', 'が高い']
...という 存在 国語 は もはや 阿 常 と 美 験 と は 思 われ ない 、 上 意 ついた 一 性を 愛 して 仲間 の 想い を ブール ジュ で 発表 するとともに ～ 勇 気 の あら いい 形で 働 いている 事例 の み だ し や 広い 米 項目 との関連 を 指摘 する 。 ▁ x x f l d ▁ 1 ▁ x x f l d ▁ 1 ▁また 、 全国大会 を終えた 後 、2008 年春 から の 一般 投票 から の 事務局 長 と 結果 が発表され 、 j ・ la ・ エ レン と おらず タイム ロス 。 編集 部 の一人 が 56 を るため に 、 さ す が に 31 世 -16 込んだ 。 ▁ x x up ▁ 営業 時間 は 8 月 23 日から 25 日の 午後 3 時に 始まり かけ 5 分 継続 して 開始された が ic b を 通り 、 その 延長 にあたった 。 ▁ x x f l d ▁ 1 ▁ x x f l d ▁ 1 ▁s ら 6 g b エンジン ▁ x x f l d ▁ 1 ▁ 近年では 現在でも 各種 の 大会 の名称 、 特別 編 制 の 特集 のほか 、 幾 十 

- 文法的に正しい単語は並んでいるようだ
- しかし、文頭のxxfld_1タグや小文字化のxxupタグが邪魔。sentencepieceの学習時と言語モデルの学習時にrulesとspecial casesを揃えるべき
    - 現状はspm学習時はrules, specialなし